In [ ]:
import bootstrap
import subprocess
import matplotlib.pyplot as plt
import time
import datetime
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
from new_ising_class import *
import math

In [2]:
mixed = MixedCorrelator()

In [3]:
exec(open("mps_results" + ".py").read())

In [4]:
len(mixed.point_table)

3417

In [5]:
def compute_n(kmax):
    return math.floor(kmax**(0.5))

keys = []
for i in range(15, 61):
    for j in range(0, 11, 1):
        if i<50:
            key = [i, i, compute_n(i)-2, compute_n(i)]
        else:
            key = [i, 50, compute_n(i)-2, compute_n(i)]
    keys.append(key)

In [6]:
keys;

In [ ]:
len(keys)

In [ ]:
mixed.grid_table[0].disallowed_points;

In [ ]:
ps = [points[0] for points in mixed.grid_table[0].disallowed_points]

In [ ]:
ps;

In [ ]:
t1= mixed.point_table[0].run_time

In [ ]:
t2 = time.strftime('%H:%M:%S', t1)

In [ ]:
t1

In [ ]:
keys[0]

In [7]:
for key in keys:
    points = [point for point in mixed.point_table if [point.kmax, point.lmax, point.mmax, point.nmax] == key]
    grid = Grid(*(key + [[], [], datetime.timedelta(hours=0, minutes=0, seconds=0), datetime.timedelta(hours=0, minutes=0, seconds=0)]))
    for point in points:
        rt1 = point.run_time
        rt2 = datetime.timedelta(hours=rt1.tm_hour, minutes=rt1.tm_min, seconds=rt1.tm_sec)
        cpu1 = point.cpu_time
        cpu2 = datetime.timedelta(hours=cpu1.tm_hour, minutes=cpu1.tm_min, seconds=cpu1.tm_sec)
        grid.run_time += rt2
        grid.cpu_time += cpu2
        if point.allowed == True:
            grid.allowed_points.append((point.sig, point.eps))
        else:
            grid.disallowed_points.append((point.sig, point.eps))
    mixed.grid_table.append(grid)

In [8]:
len(mixed.grid_table)

46

In [9]:
len(mixed.point_table)

3417

In [10]:
mixed.plot_grids(keys, "mps_results", 6, (3,2))

TypeError: Tuple or struct_time argument required

In [11]:
table = mixed.grid_table
pdf_pages = PdfPages("mps_results" + ".pdf")
nb_plots = len(table)
nb_plots_per_page = 6
nb_pages = int(np.ceil(nb_plots / float(nb_plots_per_page)))
grid_size=(3,2)
row_index = 0
for i in range(nb_plots):

    if i % nb_plots_per_page == 0:
        fig = plt.figure(figsize=(8.27, 11.69), dpi=100)
    plt.subplot2grid(grid_size, (row_index, i % grid_size[1]))
    if i % grid_size[1] == 1:
        row_index += 1
    allowed_sig = [points[0] for points in table[i].allowed_points]
    allowed_eps = [points[1] for points in table[i].allowed_points]
    disallowed_sig = [points[0] for points in table[i].disallowed_points]
    disallowed_eps = [points[1] for points in table[i].disallowed_points]
    #run_time = time.strftime('%H:%M:%S', mixed.grid_table[i].run_time)
    #cpu_time = time.strftime('%H:%M:%S', mixed.grid_table[i].cpu_time)
    plt.plot(allowed_sig, allowed_eps, 'r+')
    plt.plot(disallowed_sig, disallowed_eps, 'b+')
    plt.title('[' + table[i].kmax.__str__() + ", "
                + table[i].lmax.__str__() + ", "
                + table[i].mmax.__str__() + ", "
                + table[i].nmax.__str__() + ']'
                + "     " + str(table[i].run_time))
    if (i + 1) % nb_plots_per_page == 0 or (i + 1) == nb_plots:
        plt.tight_layout()
        pdf_pages.savefig(fig)
        row_index = 0
pdf_pages.close()